<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

In [142]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 54 (delta 30), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (54/54), 383.22 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo
dataset_original.py  EAST_dataset.ipynb        loss.py	 train.py
dataset.py	     EAST_text_detector.ipynb  model.py
CPU times: user 83.2 ms, sys: 204 ms, total: 287 ms
Wall time: 6.35 s


In [143]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [144]:
from dataset_original import custom_dataset

image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

10


In [0]:
def extract_vertices(lines):
	'''extract vertices info from txt lines
	Input:
		lines   : list of string info
	Output:
		vertices: vertices of text regions <numpy.ndarray, (n,8)>
		labels  : 1->valid, 0->ignore, <numpy.ndarray, (n,)>
	'''
	labels = []
	vertices = []
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		label = 0 if '###' in line else 1
		labels.append(label)
	return np.array(vertices), np.array(labels)

In [147]:
with open(label_address+'/gt_img_1.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)
vertices, labels

(array([[377, 117, 463, 117, 465, 130, 378, 130],
        [493, 115, 519, 115, 519, 131, 493, 131],
        [374, 155, 409, 155, 409, 170, 374, 170],
        [492, 151, 551, 151, 551, 170, 492, 170],
        [376, 198, 422, 198, 422, 212, 376, 212],
        [494, 190, 539, 189, 539, 205, 494, 206],
        [374,   1, 494,   0, 492,  85, 372,  86]]),
 array([1, 1, 0, 1, 1, 0, 0]))

In [0]:
from dataset_original import *

In [191]:
img = Image.open(image_address+'/img_1.jpg')

print(img.size)

ratio = 0.2

ratio_h = 1 + ratio * (np.random.rand() * 2 - 1)
print(ratio_h)
old_h = img.height
new_h = int(np.around(old_h * ratio_h))
img_new = img.resize((img.width, new_h), Image.BILINEAR)
print(img_new.size)

new_vertices = vertices.copy()
if vertices.size > 0:
  new_vertices[:,[1,3,5,7]] = vertices[:,[1,3,5,7]] * (new_h / old_h)
  
vertices[:,[1,3,5,7]]

(1280, 720)
0.8070327281060631
(1280, 581)


array([[-1701.12129794, -1687.66202865, -1676.74880579, -1690.36457821],
       [-1684.73363748, -1680.66455607, -1667.41428532, -1671.48336674],
       [-1670.67350893, -1665.19589933, -1652.82897997, -1658.30658957],
       [-1655.73954497, -1646.50586023, -1630.60553534, -1639.83922008],
       [-1635.02644735, -1627.82730331, -1616.34373533, -1623.54287937],
       [-1622.74253754, -1616.58324802, -1603.33297727, -1610.37561818],
       [-1797.87610808, -1779.09573233, -1709.62397934, -1727.5210037 ]])

In [178]:
scale = 0.25
length = 512

score_map   = np.zeros((int(img.height * scale), int(img.width * scale), 1), np.float32)
geo_map     = np.zeros((int(img.height * scale), int(img.width * scale), 5), np.float32)
ignored_map = np.zeros((int(img.height * scale), int(img.width * scale), 1), np.float32)

print(score_map.shape, geo_map.shape, ignored_map.shape)

index = np.arange(0, length, int(1/scale))
index_x, index_y = np.meshgrid(index, index)
ignored_polys = []
polys = []

for i, vertice in enumerate(vertices):
  print(vertice)
  print()
  poly = np.around(scale * shrink_poly(vertice).reshape((4,2))).astype(np.int32)
  polys.append(poly)
  temp_mask = np.zeros(score_map.shape[:-1], np.float32)
  cv2.fillPoly(temp_mask, [poly], 1)
  
  theta = find_min_rect_angle(vertice)
  print('theta = ',theta)
  
  rotate_mat = get_rotate_mat(theta)
  print(rotate_mat)
  rotated_vertices = rotate_vertices(vertice, theta)
  print('rotated_vertices')
  print(rotated_vertices)
  print('-'*30)

(128, 128, 1) (128, 128, 5) (128, 128, 1)
[-4258.12392856  -982.29513402 -4172.91494712  -980.89473343
 -4170.6542434   -970.28220477 -4256.85402695  -971.69888909]

theta =  -0.017453292519943295
[[ 0.9998477   0.01745241]
 [-0.01745241  0.9998477 ]]
rotated_vertices
[-4258.12392856  -982.29513402 -4172.9034845   -982.38204849
 -4170.45791094  -971.8105909  -4256.66929039  -971.72266579]
------------------------------
[-4143.23740041  -982.16954846 -4117.47654555  -981.74617154
 -4117.12767119  -968.52122004 -4142.88852604  -968.94459696]

theta =  0.0
[[ 1. -0.]
 [ 0.  1.]]
rotated_vertices
[-4143.23740041  -982.16954846 -4117.47654555  -981.74617154
 -4117.12767119  -968.52122004 -4142.88852604  -968.94459696]
------------------------------
[-4260.2822947   -951.48576503 -4225.60422086  -950.91583456
 -4225.27860478  -938.57254649 -4259.95667862  -939.14247696]

theta =  -0.017453292519943295
[[ 0.9998477   0.01745241]
 [-0.01745241  0.9998477 ]]
rotated_vertices
[-4260.2822947   -9